In [1]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
from scipy import spatial
import numpy as np
import pandas as pd

In [2]:
#load índice bienestar territorial 2012
ibt = gpd.read_file('input/geo_data/ibt/ibt_geo.shp', crs={'init': 'epsg:5361'})
ibt = ibt.dropna(subset = ['nse']).reset_index(drop = True)
ibt.tail()

,comuna,ibt,iseg,nse,geometry
40721,PUENTE ALTO,30.0,28.45,75.71,"POLYGON ((-70.54646 -33.59121, -70.54651 -33.5..."
40722,PUENTE ALTO,20.0,28.45,75.71,"POLYGON ((-70.54512 -33.59019, -70.54514 -33.5..."
40723,PUENTE ALTO,20.0,28.45,75.71,"POLYGON ((-70.54406 -33.59091, -70.54436 -33.5..."
40724,PUENTE ALTO,30.0,28.45,75.71,"POLYGON ((-70.54362 -33.59163, -70.54372 -33.5..."
40725,PUENTE ALTO,20.0,28.45,75.71,"POLYGON ((-70.54555 -33.59051, -70.54557 -33.5..."


In [3]:
# calculate the centroid of each block
mcentroids = ibt['geometry'].centroid

In [4]:
# Create KDTree to consult the closest block per point
data = list(zip(mcentroids.x.ravel(), mcentroids.y.ravel()))
tree = spatial.KDTree(data = data, leafsize = 10)

In [23]:
# Import lat lon data
georef = pd.read_csv('input/deployment_data/posts_deployment.csv', encoding='utf-8')
georef.head()

,Unnamed: 0,candidate,id_post_u,message,loc,lon,lat
0,1,ROSARIO OLIVARES,729711500563531,Hoy estuvimos en la Facultad de Arquitectura d...,"avenida portugal 84, santiago, Region Metropol...",-70.638176,-33.442279
1,2,ROSARIO OLIVARES,730476597153688,Último domingo<U+2600><U+FE0F> estuvimos en la...,"feria Santa Carolina, Macul, Region Metropolit...",-70.609463,-33.485028
2,3,ROSARIO OLIVARES,730476597153688,Último domingo<U+2600><U+FE0F> estuvimos en la...,"feria Marathon, Ñuñoa, Region Metropolitana, C...",-70.614376,-33.467316
3,4,ROSARIO OLIVARES,730476597153688,Último domingo<U+2600><U+FE0F> estuvimos en la...,"feria 10 de julio, Santiago, Region Metropolit...",-70.631530,-33.451701
4,5,ROSARIO OLIVARES,730590713808943,Terminando el último domingo de campaña tal co...,"barrio Huemul, Santiago, Region Metropolitana,...",-70.651130,-33.474624


In [24]:
# Isolate lat lon points for KDTree queries
pts = list(zip(georef['lon'].ravel(), georef['lat'].ravel()))
pts

[(-70.6381755, -33.4422791),
 (-70.6094628, -33.4850282),
 (-70.61437600000001, -33.4673157),
 (-70.6315297, -33.4517011),
 (-70.65112979999999, -33.474624),
 (-70.6201286, -33.4625335),
 (-70.580221, -33.4547576),
 (-70.66122390000001, -33.4466032),
 (-70.62093370000001, -33.4590551),
 (-70.61288470000001, -33.4598881),
 (-70.6236858, -33.556658),
 (-70.62290429999999, -33.4691839),
 (-70.6342324, -33.4627623),
 (-70.6342324, -33.4627623),
 (-70.6216513, -33.5132982),
 (-70.6342324, -33.4627623),
 (-70.6338272, -33.4675369),
 (-70.6295256, -33.5534019),
 (-70.5906488, -33.4860082),
 (-70.59222700000001, -33.475138199999996),
 (-70.59222700000001, -33.475138199999996),
 (-70.6435003, -33.4738346),
 (-70.6622487, -33.4460299),
 (-70.6106762, -33.4646281),
 (-70.6674796, -33.433673600000006),
 (-70.62337159999998, -33.4916845),
 (-70.6201286, -33.4625335),
 (-70.6201286, -33.4625335),
 (-70.6049732, -33.454179100000005),
 (-70.6649453, -33.4510363),
 (-70.6764069, -33.4326052),
 (-70.614

In [25]:
%%time
# Calculate the 2 nearest blocks per lat lon point
kind = tree.query(pts, k = 2)[1]

Wall time: 233 ms


In [26]:
%%time
# Calculate average NSE per block pair
georef.loc[:,('nse')] = list(map(lambda i: ibt.loc[kind[i],'nse'].mean(), range(len(pts))))

Wall time: 497 ms


In [27]:
#output table, with lat lon data and an associated NSE
georef.to_csv('proc/02_posts_deployment_nse.csv', encoding = 'utf-8', index = False)